# Train Stable Diffusion with CodeFlare

In [1]:
!pwd

/opt/app-root/src/teddy-demo


In [89]:
!pip install codeflare-sdk
!pip install -Ur 05_dreambooth_finetuning/dreambooth/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 136.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 171.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 159.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 169.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6
  Attempting uninstall: rich
    Found existing installation: rich 13.6.0
    Uninstalling rich-13.6.0:
      Successfully uninstalled rich-13.6.0
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.0
    Uninstalling grpcio-1.59.0:
      Successfully uninstalled grpcio-1.59.0
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.3.0
    Uninstalling charset-normalizer-3.3.0:
      Successfully uninstalled charset-normalizer-3.3.0
  Attempti

## Prep Infra

In [6]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [7]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "sha256~v3mNsdN7WLXI0ZHL90OfoOGCiJg6JB5J_BBqRYx_Dwg",
    server = "https://api.aisrhods-dims.yani.p1.openshiftapps.com:6443",
    skip_tls=False
)
auth.login()

'Logged into https://api.aisrhods-dims.yani.p1.openshiftapps.com:6443'

In [11]:
cluster = Cluster(ClusterConfiguration(
    name='stab-diff-model',
    namespace='default',
    num_workers=2,
    min_cpus=8,
    max_cpus=16,
    min_memory=4,
    max_memory=64,
    num_gpus=1,
    instascale=True, #<---instascale enabled
    machine_types=["m5.xlarge", "g5.4xlarge"],
))

Written to: stab-diff-model.yaml


In [12]:
cluster.up()

In [13]:
# cluster.down()

In [33]:
cluster.details()

                      🚀 CodeFlare Cluster Details 🚀                     
                                                                          
 ╭──────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                               │ 
 │   stab-diff-model                                        Active ✅   │ 
 │                                                                      │ 
 │   URI: ray://stab-diff-model-head-svc.default.svc:10001              │ 
 │                                                                      │ 
 │   ]8;id=563716;http://ray-dashboard-stab-diff-model-default.apps.aisrhods-dims.yani.p1.openshiftapps.com\Dashboard🔗]8;;\                                                        │ 
 │                                                                      │ 
 │                       Cluster Resources                              │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮          │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │          │ 
 │   │             │  │                                      │          │ 
 │   │  2          │  │  4~64        8           1           │          │ 
 │   │             │  │                                      │          │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯          │ 
 ╰──────────────────────────────────────────────────────────────────────╯

RayCluster(name='stab-diff-model', status=<RayClusterStatus.READY: 'ready'>, workers=2, worker_mem_min=4, worker_mem_max=64, worker_cpu=8, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-stab-diff-model-default.apps.aisrhods-dims.yani.p1.openshiftapps.com')

In [16]:
cluster.status()

╭──────────────────────────────────────────╮
│        🚀 Cluster Queue Status 🚀        │
│ +-----------------+--------------------+ │
│ | Name            | Status             | │
│ +=================+====================+ │
│ | stab-diff-model | running (starting) | │
│ |                 |                    | │
│ +-----------------+--------------------+ │
╰──────────────────────────────────────────╯

(<CodeFlareClusterStatus.STARTING: 2>, False)

In [ ]:
import codeflare_sdk as cf

In [63]:
cf.cluster.cluster.list_all_clusters('default')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_344/2159714274.py:1 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_344/2159714274.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'list_all_clusters' is not defined

## Step 0: Prep

In [17]:
!pwd

/opt/app-root/src/teddy-demo


In [51]:
# THIS IS ORIGINAL

# TODO: If running on multiple nodes, change this path to a shared directory (ex: NFS)
!export DATA_PREFIX="/opt/app-root/src/teddy-demo"
!export ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"
!export ORIG_MODEL_DIR="$DATA_PREFIX/model-orig"
!export ORIG_MODEL_PATH="$ORIG_MODEL_DIR/models--${ORIG_MODEL_NAME/\//--}/snapshots/$ORIG_MODEL_HASH"
!export TUNED_MODEL_DIR="$DATA_PREFIX/model-tuned"
!export IMAGES_REG_DIR="$DATA_PREFIX/images-reg"
!export IMAGES_OWN_DIR="$DATA_PREFIX/images-own"
!export IMAGES_NEW_DIR="$DATA_PREFIX/images-new"
# TODO: Add more worker nodes and increase NUM_WORKERS for more data-parallelism
!export NUM_WORKERS=2

# !mkdir -p 
!echo $ORIG_MODEL_DIR 
# $TUNED_MODEL_DIR $IMAGES_REG_DIR $IMAGES_OWN_DIR $IMAGES_NEW_DIR
!mkdir -p $ORIG_MODEL_DIR $TUNED_MODEL_DIR $IMAGES_REG_DIR $IMAGES_OWN_DIR $IMAGES_NEW_DIR

/model-orig
mkdir: cannot create directory ‘/model-orig’: Permission denied
mkdir: cannot create directory ‘/model-tuned’: Permission denied
mkdir: cannot create directory ‘/images-reg’: Permission denied
mkdir: cannot create directory ‘/images-own’: Permission denied
mkdir: cannot create directory ‘/images-new’: Permission denied


In [19]:
import os

In [53]:
# THIS IS ORIGINAL

# TODO: If running on multiple nodes, change this path to a shared directory (ex: NFS)
DATA_PREFIX="/opt/app-root/src/teddy-demo"

# DATA_PREFIX="/opt/app-root/src/text-to-image-demo/dim-dreambooth"
ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"
ORIG_MODEL_DIR="model-orig"
ORIG_MODEL_PATH="$ORIG_MODEL_DIR/models--${ORIG_MODEL_NAME/\//--}/snapshots/$ORIG_MODEL_HASH"
TUNED_MODEL_DIR="model-tuned"
IMAGES_REG_DIR="images-reg"
IMAGES_OWN_DIR="images-own"
IMAGES_NEW_DIR="images-new"
# TODO: Add more worker nodes and increase NUM_WORKERS for more data-parallelism
!export NUM_WORKERS=2

os.makedirs(ORIG_MODEL_DIR, exist_ok=True)
os.makedirs(TUNED_MODEL_DIR, exist_ok=True)
os.makedirs(IMAGES_REG_DIR, exist_ok=True)
os.makedirs(IMAGES_OWN_DIR, exist_ok=True)
os.makedirs(IMAGES_NEW_DIR, exist_ok=True)

In [33]:
# !export DATA_PREFIX="/opt/app-root/src/text-to-image-demo/dim-dreambooth"
# !export ORIG_MODEL_DIR="$DATA_PREFIX/model-orig"
# !export ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
# !export ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"


In [54]:
!echo $DATA_PREFIX
!echo $ORIG_MODEL_NAME
!echo $ORIG_MODEL_HASH

/opt/app-root/src/teddy-demo
CompVis/stable-diffusion-v1-4
b95be7d6f134c3a9e62ee616f310733567f069ce


## Step 1: Download the pre-trained model

In [23]:
!pwd

/opt/app-root/src/teddy-demo


In [32]:
# !python 05_dreambooth_finetuning/dreambooth/cache_model.py --model_dir=$DATA_PREFIX/images-reg --model_name=$ORIG_MODEL_NAME --revision=$ORIG_MODEL_HASH

## Step 2: Supply images of your subject

In [56]:
  # Only uncomment one of the following:

  # Option 1: Use the dog dataset ---------
!export CLASS_NAME="dog"
# !python 05_dreambooth_finetuning/dreambooth/download_example_dataset.py ./images/dog
CLASS_NAME='dog'
  # export INSTANCE_DIR=./images/dog
  # ---------------------------------------

  # Option 2: Use the lego car dataset ----
 # !export CLASS_NAME="car"
 # !export INSTANCE_DIR=./images/lego-car
  # ---------------------------------------

  # Option 3: Use your own images ---------
  # export CLASS_NAME="<class-of-your-subject>"
  # export INSTANCE_DIR="/path/to/images/of/subject"
  # ---------------------------------------

  # Copy own images into IMAGES_OWN_DIR
# !cp -rf $INSTANCE_DIR/* "$IMAGES_OWN_DIR/"

## Step 3: Create the regularization images

In [55]:
!echo $CLASS_NAME

In [31]:
!pip install flags

  Preparing metadata (setup.py) ... done
  Created wheel for flags: filename=flags-0.0.1.2-py3-none-any.whl size=1698 sha256=3803f64bf6424a9f7380aebf4f6b56a5af600181899edcffd42107cfb5577489
  Stored in directory: /tmp/pip-ephem-wheel-cache-_3zwiw96/wheels/70/d5/c6/b10a3579bc3ccbcab5fb92dfbb8b16e075cd86dab671fce1a1
Successfully built flags

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [30]:
!pip install pyarrow


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [57]:
!python 05_dreambooth_finetuning/dreambooth/generate.py \
    --model_dir=ORIG_MODEL_PATH \
    --output_dir=IMAGES_REG_DIR \
    --prompts=f"photo of a dog" \
    --num_samples_per_prompt=200 \
    --use_ray_data

2023-10-08 10:07:10,981	INFO packaging.py:518 -- Creating a file package for local directory '.'.
2023-10-08 10:07:10,987	WARNING packaging.py:393 -- File /opt/app-root/src/teddy-demo/images-reg/models--CompVis--stable-diffusion-v1-4/snapshots/b95be7d6f134c3a9e62ee616f310733567f069ce/safety_checker/model.fp16.safetensors is very large (579.85MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/opt/app-root/src/teddy-demo/images-reg/models--CompVis--stable-diffusion-v1-4/snapshots/b95be7d6f134c3a9e62ee616f310733567f069ce/safety_checker/model.fp16.safetensors']})`
2023-10-08 10:07:12,422	WARNING packaging.py:393 -- File /opt/app-root/src/teddy-demo/images-reg/models--CompVis--stable-diffusion-v1-4/snapshots/b95be7d6f134c3a9e62ee616f310733567f069ce/safety_checker/pytorch_model.fp16.bin is very large (579.93MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': [

## Fine tune the model

In [43]:
!pip install ray['train'] --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 158.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 125.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 171.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 135.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 147.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 174.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 148.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 156.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 167.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 130.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 156.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 150.8 MB/s eta 0:00:00a 0:00:01
 

In [ ]:
UNIQUE_TOKEN = 'unqtkn'
CLASS_NAME = 'dog'

In [62]:
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=cddb6c5d69c444075c5c3090b592463eb4333c7c948c6bd36c768a52020403e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-hmeq78fu/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [91]:
!pip install ray['rllib'] --force-reinstall
!pip install ray['Train'] --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 152.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 178.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 153.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 137.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 171.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 144.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 172.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 147.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 154.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 176.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 177.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 155.4 MB/s eta 0:00:00a 0:00:01
 

In [92]:
!pip install ray['Air'] --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 161.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 158.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 123.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 179.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 153.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 173.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 164.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 171.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 179.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 157.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 165.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 174.8 MB/s eta 0:00:00
   

In [83]:
!pip install xformers --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 166.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 162.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 163.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 167.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 179.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 163.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 149.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 165.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 162.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 171.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 168.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [87]:
!pip install grpcio --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 96.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.59.0
    Uninstalling grpcio-1.59.0:
      Successfully uninstalled grpcio-1.59.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.33.2 requires protobuf<4.0dev,>=3.5.0.post1, but you have protobuf 4.24.4 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [78]:
!pip install -r teddy-demo/requirements3.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'teddy-demo/requirements3.txt'

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [99]:
!pip install pyarrow --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 155.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 151.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 13.0.0
    Uninstalling pyarrow-13.0.0:
      Successfully uninstalled pyarrow-13.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.14.5 requires fsspec[http]<2023.9.0,>=2023.1.0, but you have fsspec 2023.9.2 which is incompatible.
codeflare-sdk 0.8.0 requires ray[default]==2.5.0, but you have ray 2.7.0 which is incompatible.
codeflare-sdk 0.8.0 requires rich<13.0,>=12.5, but you have rich 13.6.0 which is incompatible.

[notice] A new release of pip av

In [103]:
!python 05_dreambooth_finetuning/dreambooth/train.py \
    --model_dir=images-reg \
    --output_dir=model-tuned \
    --instance_images_dir=images-own \
    --instance_prompt="photo of unqtkn dog" \
    --class_images_dir=images-reg \
    --class_prompt="photo of a dog" \
    --train_batch_size=2 \
    --lr=5e-6 \
    --num_epochs=4 \
    --max_train_steps=200 \
    --num_workers=2

Namespace(model_dir='images-reg', output_dir='model-tuned', use_lora=False, instance_images_dir='images-own', instance_prompt='photo of unqtkn dog', class_images_dir='images-reg', class_prompt='photo of a dog', train_batch_size=2, lr=5e-06, num_epochs=4, max_train_steps=200, prior_loss_weight=1.0, max_grad_norm=1.0, num_workers=2)
2023-10-08 12:01:42,797	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-10-08 12:01:44,420	INFO read_api.py:406 -- To satisfy the requested parallelism of 28, each read task output is split into 2 smaller blocks.
2023-10-08 12:01:44,447	INFO read_api.py:406 -- To satisfy the requested parallelism of 28, each read task output is split into 28 smaller blocks.
2023-10-08 12:01:44,609	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage->SplitBlocks(2)] -> TaskPoolMapOperator[MapBatches(<lambda>)->MapBatches(transform_image)->MapBatches(<lambda>)->MapBatches(process_

In [ ]:
cluster.details()

In [ ]:
from codeflare_sdk.job.jobs import DDPJobDefinition

In [ ]:
arg_list = [
    "--model_name_or_path", "gpt2",
    "--dataset_name", "wikitext",
    "--dataset_config_name", "wikitext-2-raw-v1",
    "--per_device_train_batch_size", "2",
    "--per_device_eval_batch_size", "2",
    "--do_train",
    "--do_eval",
    "--output_dir", "/tmp/test-clm",
    "--overwrite_output_dir"
]

In [ ]:
jobdef = DDPJobDefinition(
    name="gpttest",
    script="gpt_og.py",
    script_args=arg_list,
    scheduler_args={"requirements": "requirements_gpt.txt"}
)
job = jobdef.submit(cluster)

In [ ]:
job.status()

Retrieve raw log output at anytime with:

In [ ]:
job.logs()

View live updates for status, logs, and other information with:

In [ ]:
cluster.cluster_dashboard_uri()

In [ ]:
job.status()

In [ ]:
cluster.down()

In [ ]:
auth.logout()